<a href="https://colab.research.google.com/github/Hamza-Ali0237/DC-Analyze-Car-Reviews-HuggingFace/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing Car Reviews with LLMs (HuggingFace)

In [38]:
!pip install evaluate

In [39]:
import torch
import evaluate
import pandas as pd
from transformers import pipeline

In [40]:
df = pd.read_csv("car_reviews.csv", delimiter=";")
df.head()

,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE
1,The car is fine. It's a bit loud and not very ...,NEGATIVE
2,"My first foreign car. Love it, I would buy ano...",POSITIVE
3,I've come across numerous reviews praising the...,NEGATIVE
4,I've been dreaming of owning an SUV for quite ...,POSITIVE


In [41]:
reviews = df["Review"].tolist()
labels = df['Class'].tolist()

# Classification

In [42]:
classifier = pipeline(task="sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

Device set to use cuda:0


In [43]:
predicted_labels = classifier(reviews)

In [44]:
references = [1 if label == "POSITIVE" else 0 for label in labels]
predictions = [1 if label['label'] == "POSITIVE" else 0 for label in predicted_labels]

In [45]:
print(predictions)

[1, 1, 1, 0, 1]


In [46]:
accuracy = evaluate.load("accuracy")

In [47]:
f1 = evaluate.load("accuracy")

In [48]:
accuracy_result = accuracy.compute(references=references, predictions=predictions)
accuracy_result = accuracy_result['accuracy']

In [49]:
f1_result = f1.compute(references=references, predictions=predictions)
f1_result = f1_result['accuracy']

# Translation

In [50]:
translator = pipeline(task="translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


In [51]:
translation = translator(reviews[0], max_length=27)
translated_review = translation[0]['translation_text']
print(translated_review)

Your input_length: 365 is bigger than 0.9 * max_length: 27. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


Estoy muy satisfecho con mi 2014 Nissan NV SL. Uso esta furgoneta para mis entregas de negocios y uso personal.


In [52]:
bleu = evaluate.load("bleu")

In [53]:
with open("reference_translations.txt", 'r') as file:
    lines = file.readlines()
references_2 = [line.strip() for line in lines]

In [54]:
bleu_score = bleu.compute(predictions=[translated_review], references=[references_2])
print(bleu_score)

{'bleu': 0.6022774485691839, 'precisions': [0.9090909090909091, 0.7142857142857143, 0.55, 0.3684210526315789], 'brevity_penalty': 1.0, 'length_ratio': 1.0476190476190477, 'translation_length': 22, 'reference_length': 21}


In [55]:
bleu_score = bleu_score["bleu"]

In [56]:
print(bleu_score)

0.6022774485691839
